In [1]:
#  Import libraries and define variables
import os
import shutil
import glob
from tqdm import tqdm
from PIL import Image
import nibabel as nib
import numpy as np


classifications = ['benign', 'malignant']

# Define patch size and stride
block_h, block_w = 64, 64
stride = 2

# Interpolated image dimestions
inter_dim = (110, 90)

# Define paths to the BraTS dataset folders
path = 'F:/gliobastoma-mgmt-upenn/Data/Skin Cancer'

# PATH = path + 'Data/BRATS/novel_data/'
Org_Dir = path + '/Original_Data_Backup/'
Work_Dir = path + '/Working_Data/'

In [3]:
# Arbitrary functions

def remove_items_starting_with(lst, start_str):
    """
    Remove items from a list that start with the given string.

    Args:
        lst (list): The list from which to remove items.
        start_str (str): The string that items should start with to be removed.

    Returns:
        list: The modified list with items removed.
    """
    # Create a list to store items to be removed
    items_to_remove = []

    # Iterate over the list
    for item in lst:
        if item.startswith(start_str):
            items_to_remove.append(item)

    # Remove items from the list
    for item in items_to_remove:
        lst.remove(item)

    return lst


In [25]:
 # Function Definition --> Backup() Backup Original Data For Safety Purpose

def Backup():
    import shutil
    print('Creating Backup')
    # Copy data for backup
    try:
        shutil.copytree(path, Org_Dir)
    except Exception as e:
        print(e)

    # Copy data for working
    try:
        shutil.copytree(Org_Dir, Work_Dir)
    except Exception as e:
        print(e)

    # Deleting folder, listed in the CATEGORIES list, after creating Cases
    for cate in classifications:
        try:
            shutil.rmtree(path + cate)  # Deleting Folders of CATEGORIES list
        except Exception as e:
            print(e)
    print('Backup Created')

In [27]:
#  Function Definition -- reconstruct() Reconstruction of Working Directory
def reconstruct():
    import shutil
    print('Reconstructing')
    # Deleting working directory
    shutil.rmtree(path + "Working_data/")
    # Copying data from backup
    try:
        shutil.copytree(Org_Dir, Work_Dir)
    except:
        print('Working directory already exists')

    print('Reconstruction complete')

In [2]:
# Function Defination -> Interpolation() Interpolate Images

def Interpolation():
    print('Interpolating...')
    try:
        workdir = os.listdir(Work_Dir)
        
        for label in workdir:
            label_path = os.path.join(Work_Dir, label + '/') # Ex: Work_dir + benign/ 
            img_file = os.listdir(label_path)   # Gives the list of Images in benign or malignant folders 
            os.chdir(label_path)
            for img in img_file:
                image = Image.open(img)
                image = image.resize(inter_dim, Image.ANTIALIAS)
                inter_img_path = 'inter_' + img
                image.save(inter_img_path)
                image.close()
                os.remove(img)
                
    except Exception as e:
        print('Error in Interpolation() function')
        print(e)

In [3]:
Interpolation()

Interpolating...


C:\Users\J Narasimha Rao\AppData\Local\Temp\ipykernel_14980\3095380726.py:14: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize(inter_dim, Image.ANTIALIAS)


In [4]:
#  Function Defination -> Creating_patches() function

def Creating_patches(block_h, block_w, stride):
    print('Creating patches...')
    try:
        workdir = os.listdir(Work_Dir)  # Work Directory
        for label in workdir:
            label_path = os.path.join(Work_Dir, label + '/')    # Working_dir/benign/
            img_file = os.listdir(label_path)  # File list of Working_dir/benign/
            os.chdir(label_path)
            for img in tqdm(glob.glob('*.jpg')):
                image = Image.open(img)
                img_w, img_h = image.size

                File_Name, extention = os.path.splitext(img)

                try:
                    os.mkdir(File_Name)
                except Exception as e:
                    print('Error in Creating_patches() making directory')
                    print(e)
                    
                frame_num = 0
                count_row = 0

                for row in range(0, img_h, stride):
                    if (img_h-row >= block_h):
                        count_row += 1
                        count_col = 0

                        for col in range(0, img_w, stride):
                            if ( img_h -col >= block_w ):
                                count_col += 1
                                frame_num += 1

                                box = (col, row, col+block_w, row+block_h)
                                a = image.crop(box)
                                a.save(os.path.join(File_Name,File_Name + '_row_' + str(count_row) + '_col_' + str(count_col) + '.jpg'))
                
                image.close()
                os.remove(img)
                
            
    except Exception as e:
        print('Error in Creating_patches() function')
        print(e)

In [5]:
Creating_patches(block_h, block_w, stride)

Creating patches...


100%|██████████| 48/48 [00:09<00:00,  4.97it/s]


In [6]:
# Function Defination --> Pooling Patches in a single folder

def Pooling_patches():
    print('Pooling patches...')
    try:
        workdir = os.listdir(Work_Dir) 
        
        for label in workdir:
            label_path = os.path.join(Work_Dir, label + '/')    # Work_Dir/benign
            interplot_folder = os.listdir(label_path) # List of images in Work_Dir/benign
            os.chdir(label_path)
            for inter in tqdm(interplot_folder):
                img_file=os.listdir(inter)
                for img in img_file:
                    img_path=os.path.join(inter,img)
                    shutil.move(img_path,label_path)
                    
                os.rmdir(inter)
            
            

    except Exception as e:
        print('Error in Pooling_patches() function')
        print(e)

In [7]:
Pooling_patches()

Pooling patches...


100%|██████████| 48/48 [00:18<00:00,  2.53it/s]


In [6]:
# Main cell to invoke functions

# Backup the original data
# Backup()

# reconstruct the working data
# reconstruct()

# create modality folders
# create_modality_folders()

# Generate images from nifti files
# Generate_images()

# # Interpolate the images
# Interpolation()

# Creating patches
# Creating_patches(block_h, block_w, stride)

# Pooling patches
Pooling_patches()

Pooling patches...


100%|██████████| 3/3 [16:08<00:00, 322.78s/it]
